In [2]:
import tensorflow as tf
import keras
import numpy as np
import glob
import csv
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
lines = []
with(open('./data/driving_log.csv')) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        #if line.split("/")[1].split("_")[0] == 'center':
        lines.append(line)

# Removing the column title row
lines = lines[1:]

In [4]:
# Initiating a blank dictionary for data
data = {'train':[], 'validation':[]}
for line in lines:
    # Reading the steering measurement
    measurement = float(line[3])
    # Removing about 95% of the images with measurement = 0
    if measurement != 0 or np.random.rand() > 0.95:
        # loop 3 times for three different positions of left, center, right
        for i in range(3):
            datapoint = {}
            datapoint['path'] = './data/' + line[i].lstrip()
            datapoint['position'] = line[i].split('/')[1].split('_')[0]
            datapoint['flipped'] = False
            # Adding or subtracting 5.0 (can be tuned) to the steering angle based on the camera position
            if datapoint['position'] == 'right':                
                datapoint['measurement'] = measurement - 5.0
            elif datapoint['position'] == 'left':
                datapoint['measurement'] = measurement + 5.0
            else:
                datapoint['measurement'] = measurement
            if datapoint['position'] == 'center' and np.random.rand() > 0.8:
                data['validation'].append(datapoint)
            else:
                data['train'].append(datapoint)
            # Creating a copy of image dictionary to avoid pointing to the previous one
            datapoint = dict(datapoint)    
            # Augmenting with the flipped image
            datapoint['flipped'] = True
            datapoint['measurement'] = -datapoint['measurement']
            data['train'].append(datapoint)

In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, samples, batch_size=128, dim=(160,320), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.samples = samples
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.samples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_datapoints_temp = [self.samples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_datapoints_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.samples))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_datapoints_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=float)

        # Generate data
        for i, datapoint in enumerate(list_datapoints_temp):
            image = plt.imread(datapoint['path'])
            # flipping horizontal if necessary
            if datapoint['flipped']:
                image = cv2.flip(image, 1)
            # Store sample
            X[i,] = image

            # Store value
            y[i] = datapoint['measurement']

        return X, y

In [6]:
params = {'dim': (160,320), 
          'batch_size': 128, 
          'n_channels': 3,
          'shuffle': True}

In [7]:
training_generator = DataGenerator(data['train'], **params)
validation_generator = DataGenerator(data['validation'], **params)

In [8]:
training_generator.__getitem__(0)[0][0].shape

(160, 320, 3)

In [9]:
def image_preprocess(images):
    images = tf.image.rgb_to_yuv(images)
    images = tf.image.crop_to_bounding_box(images, 60, 0, 80, 320)
    images = tf.image.resize(images, [66,200])
    images = tf.math.add(tf.math.divide(images,128), -1)
    return images

In [10]:
# CNN Architecture
## Setup Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D
from keras.layers import Lambda
from keras.layers import Input

In [11]:
# Building DAVE2 Architecture in Keras 
model = Sequential()

# preprocessing
#model.add(Lambda(image_preprocess, input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/128.0 - 1.0, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((60,20),(0,0))))

model.add(Conv2D(24, (5,5), strides=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(36, (5,5), strides=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(48, (5,5), strides=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('tanh'))

In [12]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 80, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 38, 158, 24)       1824      
_________________________________________________________________
activation_1 (Activation)    (None, 38, 158, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 77, 36)        21636     
_________________________________________________________________
activation_2 (Activation)    (None, 17, 77, 36)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 37, 48)        

In [13]:
model.compile(loss='mse', optimizer='adam')
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator, 
                    epochs=10,
                    use_multiprocessing=True,
                    workers=6)
model.save('model.h5')


Epoch 1/10
176/176 [==============================] - 106s 604ms/step - loss: 15.9619 - val_loss: 0.8316
Epoch 2/10
176/176 [==============================] - 103s 587ms/step - loss: 15.0153 - val_loss: 0.9124
Epoch 3/10
176/176 [==============================] - 101s 575ms/step - loss: 14.1946 - val_loss: 0.9088
Epoch 4/10
176/176 [==============================] - 105s 597ms/step - loss: 14.1767 - val_loss: 0.8557
Epoch 5/10
176/176 [==============================] - 107s 609ms/step - loss: 13.9075 - val_loss: 0.8806
Epoch 6/10
176/176 [==============================] - 101s 576ms/step - loss: 13.9177 - val_loss: 0.8586
Epoch 7/10
176/176 [==============================] - 102s 578ms/step - loss: 13.7414 - val_loss: 0.8846
Epoch 8/10
176/176 [==============================] - 97s 549ms/step - loss: 13.6315 - val_loss: 0.9005
Epoch 9/10
176/176 [==============================] - 93s 527ms/step - loss: 13.7836 - val_loss: 0.8978
Epoch 10/10
176/176 [==============================] - 9